## Naive Bayes on Political Text

In this notebook we use Naive Bayes to explore and classify political data. See the `README.md` for full details. You can download the required DB from the shared dropbox or from blackboard

In [20]:
import sqlite3
import pandas as pd
import nltk
import random
import numpy as np
from collections import Counter, defaultdict
from string import punctuation
import re
from nltk.corpus import stopwords

# Feel free to include your text patterns functions
#from text_functions_solutions import clean_tokenize, get_patterns

In [5]:
convention_db = sqlite3.connect("2020_Conventions.db")
convention_cur = convention_db.cursor()

### Part 1: Exploratory Naive Bayes

We'll first build a NB model on the convention data itself, as a way to understand what words distinguish between the two parties. This is analogous to what we did in the "Comparing Groups" class work. First, pull in the text 
for each party and prepare it for use in Naive Bayes.  

In [15]:
convention_cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(convention_cur.fetchall())

[('conventions',)]


In [18]:
#first look at what the table in the DF looks like
col = pd.DataFrame(convention_cur.execute("""SELECT * FROM conventions"""))
col.head()

,0,1,2,3,4,5,6,7
0,Democratic,4,Unknown,1,00:00,Skip to content The Company Careers Press Free...,127,www_rev_com_blog_transcripts2020-democratic-na...
1,Democratic,4,Speaker 1,1,00:33,I’m here by calling the full session of the 48...,41,www_rev_com_blog_transcripts2020-democratic-na...
2,Democratic,4,Speaker 2,1,00:59,"Every four years, we come together to reaffirm...",17,www_rev_com_blog_transcripts2020-democratic-na...
3,Democratic,4,Kerry Washington,1,01:07,We fight for a more perfect union because we a...,28,www_rev_com_blog_transcripts2020-democratic-na...
4,Democratic,4,Bernie Sanders,1,01:18,"We must come together to defeat Donald Trump, ...",22,www_rev_com_blog_transcripts2020-democratic-na...


In [21]:
convention_data = []

# fill this list up with items that are themselves lists. The 
# first element in the sublist should be the cleaned and tokenized
# text in a single string. The second element should be the party. 

query_results = convention_cur.execute(
                            '''
                            SELECT text, party FROM conventions
                            ''')

for row in query_results :
    # store the results in convention_data
    
    text, party = row
    
    # The  first element in the sublist should be the cleaned 
    # and tokenized text in a single string
    
    #********* Removing the punctuations ***************
    punctuation = set(punctuation) # creating punctuation Set
    more_punct_elements = ["›", "«", "×"]
    punctuation.update(more_punct_elements) # updating our punctuation sets
    
    
    text_no_punct = "".join([ch for ch in text if ch not in punctuation])
    
    #********* Removing the numbers ***************
    
    text_no_numbers = "".join([ch for ch in text_no_punct if not ch.isdigit()])
    
    #********* Tokenizing ***************
    whitespace_pattern = re.compile(r"\s+")
    
    tokens = [item.lower() for item in whitespace_pattern.split(text_no_numbers)]
    
    #*********** Removing Stopwords **************
    sw = stopwords.words("english") # stopwords
    
    tokens_no_sw = [token for token in tokens if not token in sw]
    
    #********* From tokenized into  single string *************
    
    # tokenized text in a single string
    
    text_tokenized = " ".join(tokens_no_sw)
    
    #********* Appending to convention_data ***************
    # The second element should be the party
    
    convention_data.append([text_tokenized, party])
    
# print(convention_data)
    


Let's look at some random entries and see if they look right. 

In [22]:
random.choices(convention_data,k=10)

[['behalf united states', 'Republican'],
 ['idaho', 'Democratic'],
 ['times challenging believe there’s one way forward united america united america united pursuit perfect union united dreams better future us children united determination make coming years bright ready believe great nation we’re good decent people lord’s sake united states america there’s never anything we’ve able accomplish we’ve done together',
  'Democratic'],
 ['immediately upon taking office president trump changed things change threatened establishment establishment fought back democrat obstruction phony investigations dishonest media incredible stories negative president seen anything like despite everything threw president trump delivered american people delivered like never building strongest economy american history million new jobs lowest ever unemployment black hispanic americans ending biden era lopsided trade deals sent factories overseas passing historic usmca taking china winning trade war protecting s

If that looks good, we now need to make our function to turn these into features. In my solution, I wanted to keep the number of features reasonable, so I only used words that occur at least `word_cutoff` times. Here's the code to test that if you want it. 

In [23]:
word_cutoff = 5

tokens = [w for t, p in convention_data for w in t.split()]

word_dist = nltk.FreqDist(tokens)

feature_words = set()

for word, count in word_dist.items() :
    if count > word_cutoff :
        feature_words.add(word)
        
print(f"With a word cutoff of {word_cutoff}, we have {len(feature_words)} as features in the model.")

With a word cutoff of 5, we have 2338 as features in the model.


In [24]:
def conv_features(text,fw) :
    """Given some text, this returns a dictionary holding the
       feature words.
       
       Args: 
            * text: a piece of text in a continuous string. Assumes
            text has been cleaned and case folded.
            * fw: the *feature words* that we're considering. A word 
            in `text` must be in fw in order to be returned. This 
            prevents us from considering very rarely occurring words.
        
       Returns: 
            A dictionary with the words in `text` that appear in `fw`. 
            Words are only counted once. 
            If `text` were "quick quick brown fox" and `fw` = {'quick','fox','jumps'},
            then this would return a dictionary of 
            {'quick' : True,
             'fox' :    True}
        
    """
    
    # Your code here
    
    ret_dict = dict()

    for i in text.split():
    
        if i in fw:

            ret_dict[i] = True
    
    return(ret_dict)

In [25]:
assert(len(feature_words)>0)
assert(conv_features("donald is the president",feature_words)==
       {'donald':True,'president':True})
assert(conv_features("people are american in america",feature_words)==
                     {'america':True,'american':True,"people":True})

Now we'll build our feature set. Out of curiosity I did a train/test split to see how accurate the classifier was, but we don't strictly need to since this analysis is exploratory. 

In [26]:
featuresets = [(conv_features(text,feature_words), party) for (text, party) in convention_data]

In [27]:
random.seed(20220507)
random.shuffle(featuresets)

test_size = 500

In [28]:
test_set, train_set = featuresets[:test_size], featuresets[test_size:]
classifier = nltk.NaiveBayesClassifier.train(train_set)
print(nltk.classify.accuracy(classifier, test_set))

0.496


In [29]:
classifier.show_most_informative_features(25)

Most Informative Features
                   china = True           Republ : Democr =     25.8 : 1.0
                   votes = True           Democr : Republ =     23.8 : 1.0
             enforcement = True           Republ : Democr =     21.5 : 1.0
                 destroy = True           Republ : Democr =     19.2 : 1.0
                freedoms = True           Republ : Democr =     18.2 : 1.0
                 climate = True           Democr : Republ =     17.8 : 1.0
                supports = True           Republ : Democr =     17.1 : 1.0
                   crime = True           Republ : Democr =     16.1 : 1.0
                   media = True           Republ : Democr =     14.9 : 1.0
                 beliefs = True           Republ : Democr =     13.0 : 1.0
               countries = True           Republ : Democr =     13.0 : 1.0
                 defense = True           Republ : Democr =     13.0 : 1.0
                    isis = True           Republ : Democr =     13.0 : 1.0

Write a little prose here about what you see in the classifier. Anything odd or interesting?

### My Observations

The largest differences in tokens used by republicans vs democrats is the word China.  This at a rate of almost 25:1.  also interesting is the fact the word "destroy" shows up at a rate of 20:1.  The context of these words would be very interesting to explore.



## Part 2: Classifying Congressional Tweets

In this part we apply the classifer we just built to a set of tweets by people running for congress
in 2018. These tweets are stored in the database `congressional_data.db`. That DB is funky, so I'll
give you the query I used to pull out the tweets. Note that this DB has some big tables and 
is unindexed, so the query takes a minute or two to run on my machine.

In [30]:
cong_db = sqlite3.connect("congressional_data.db")
cong_cur = cong_db.cursor()

In [31]:
results = cong_cur.execute(
        '''
           SELECT DISTINCT 
                  cd.candidate, 
                  cd.party,
                  tw.tweet_text
           FROM candidate_data cd 
           INNER JOIN tweets tw ON cd.twitter_handle = tw.handle 
               AND cd.candidate == tw.candidate 
               AND cd.district == tw.district
           WHERE cd.party in ('Republican','Democratic') 
               AND tw.tweet_text NOT LIKE '%RT%'
        ''')

results = list(results) # Just to store it, since the query is time consuming

In [32]:
tweet_data = []

# Now fill up tweet_data with sublists like we did on the convention speeches.
# Note that this may take a bit of time, since we have a lot of tweets.

for row in results:
    # store the results in convention_data
    
    candidate, party, text = row
    
    # The  first element in the sublist should be the cleaned 
    # and tokenized text in a single string
    
    #************* Decoding the Bytes ******************
    text_decoded = text.decode('utf-8')
    
    #********* Removing the punctuations ***************
    punctuation = set(punctuation) # creating punctuation Set
    more_punct_elements = ["›", "«", "×"]
    punctuation.update(more_punct_elements) # updating our punctuation sets
    
    
    text_no_punct = "".join([ch for ch in text_decoded if ch not in punctuation])
    
    #********* Removing the numbers ***************
    
    text_no_numbers = "".join([ch for ch in text_no_punct if not ch.isdigit()])
    
    #********* Tokenizing ***************
    whitespace_pattern = re.compile(r"\s+")
    
    tokens = [item.lower() for item in whitespace_pattern.split(text_no_numbers)]
    
    #*********** Removing Stopwords **************
    sw = stopwords.words("english") # stopwords
    
    tokens_no_sw = [token for token in tokens if not token in sw]
    
    #********* From tokenized into  single string *************
    
    # tokenized text in a single string
    
    text_tokenized = " ".join(tokens_no_sw)
    
    #********* Appending to convention_data ***************
    # The second element should be the party
    
    tweet_data.append([text_tokenized, party])

There are a lot of tweets here. Let's take a random sample and see how our classifer does. I'm guessing it won't be too great given the performance on the convention speeches...

In [33]:
random.seed(20201014)

tweet_data_sample = random.choices(tweet_data,k=10)

In [34]:
for tweet, party in tweet_data_sample :
    featureset = conv_features(tweet, feature_words)
    
    estimated_party = classifier.classify(featureset)
    # Fill in the right-hand side above with code that estimates the actual party
    
    print(f"Here's our (cleaned) tweet: {tweet}")
    print(f"Actual party is {party} and our classifer says {estimated_party}.")
    print("")
    

Here's our (cleaned) tweet: earlier today spoke house floor abt protecting health care women praised ppmarmonte work central coast httpstcowqgtrztvv
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: go tribe rallytogether httpstconxutfll
Actual party is Democratic and our classifer says Democratic.

Here's our (cleaned) tweet: apparently trump thinks easy students overwhelmed crushing burden debt pay student loans trumpbudget httpstcockyqotqh
Actual party is Democratic and our classifer says Republican.

Here's our (cleaned) tweet: we’re grateful first responders rescue personnel firefighters police volunteers working tirelessly keep people safe provide muchneeded help putting lives line httpstcoezpvvmiz
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: let’s make even greater kag 🇺🇸 httpstcoyqozdlz
Actual party is Republican and our classifer says Republican.

Here's our (cleaned) tweet: hr cavs tie seri

Now that we've looked at it some, let's score a bunch and see how we're doing.

In [39]:
# dictionary of counts by actual party and estimated party. 
# first key is actual, second is estimated
parties = ['Republican','Democratic']
results = defaultdict(lambda: defaultdict(int))

for p in parties :
    for p1 in parties :
        results[p][p1] = 0


num_to_score = 10000
random.shuffle(tweet_data)

for idx, tp in enumerate(tweet_data) :
    tweet, party = tp    
    # Now do the same thing as above, but we store the results rather
    # than printing them. 
   
    featureset = conv_features(tweet, feature_words)

    # get the estimated party
    estimated_party = classifier.classify(featureset)
    
    results[party][estimated_party] += 1
    
    if idx > num_to_score : 
        break

In [40]:
results

defaultdict(<function __main__.<lambda>()>,
            {'Republican': defaultdict(int,
                         {'Republican': 3743, 'Democratic': 629}),
             'Democratic': defaultdict(int,
                         {'Republican': 4779, 'Democratic': 851})})

### Reflections

The classifier looked at 10000 different tweets, in an attempt to classify them as either republican or democrat.  It seems clear that this classifier is much better at detecting republicans vice democrats.  This could be because of poor or missing labels for tweets related to democrats.